# Neural Network Model with Pytorch

In this notebook we will build a neural network model with Pytorch library. The model will consist of 3 Dense connected layer and one output layer that map to the number of classes need to classify (in this case is 3)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("fetal_health.csv")
X = data.drop(columns=['fetal_health']).values
y = data['fetal_health'].values - 1

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [3]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x
        
input_size = X_train.shape[1]
hidden_size = 512
num_classes = len(np.unique(y))
model = NeuralNet(input_size, hidden_size, num_classes)

print(model)

NeuralNet(
  (relu): ReLU()
  (fc1): Linear(in_features=21, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=3, bias=True)
)


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [5]:
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/20], Loss: 1.3351
Epoch [2/20], Loss: 0.4079
Epoch [3/20], Loss: 0.3663
Epoch [4/20], Loss: 0.4452
Epoch [5/20], Loss: 0.3390
Epoch [6/20], Loss: 0.3504
Epoch [7/20], Loss: 0.3673
Epoch [8/20], Loss: 0.3464
Epoch [9/20], Loss: 0.3423
Epoch [10/20], Loss: 0.3108
Epoch [11/20], Loss: 0.3229
Epoch [12/20], Loss: 0.3049
Epoch [13/20], Loss: 0.3757
Epoch [14/20], Loss: 0.3234
Epoch [15/20], Loss: 0.3443
Epoch [16/20], Loss: 0.3558
Epoch [17/20], Loss: 0.3154
Epoch [18/20], Loss: 0.3095
Epoch [19/20], Loss: 0.3262
Epoch [20/20], Loss: 0.2973


In [6]:
model.eval()
y_pred_list = []
with torch.no_grad():
    for X_batch, _ in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_pred_list.append(predicted)

y_pred = torch.cat(y_pred_list).cpu().numpy()
y_true = y_test_tensor.cpu().numpy()

# Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
class_report = classification_report(y_true, y_pred)

print("Neural Network Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nClassification Report:")
print(class_report)

Neural Network Model Evaluation Metrics:
Accuracy: 0.85
Precision: 0.83
Recall: 0.85
F1 Score: 0.83

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.98      0.92       332
           1       0.57      0.27      0.37        59
           2       0.95      0.60      0.74        35

    accuracy                           0.85       426
   macro avg       0.80      0.62      0.67       426
weighted avg       0.83      0.85      0.83       426

